In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import uproot

from tqdm.notebook import tqdm


# Loading MiniBooNE MC File

In [ ]:
# downloaded from /pnfs/uboone/persistent/users/markross/MiniBooNEDatasets2023/
f = uproot.open("data_files/output_osc_mc_detail_1.root")

print(f["MiniBooNE_CCQE"].keys())


In [ ]:
all_mc_pot = 41.10e20 # POT, from SomeMiniBooNEDetails_on_files_v2.pdf
data_pot = 18.75e20 # POT

# approximate! looking at file sizes rather than numbers of events
curr_file_size = 4398175
all_file_sizes = [
    1836902,
    4398175,
    4388901,
    4402467,
    4397328,
    4399516,
    4394821,
    4397983,
    4401235,
    4394093,
]

curr_mc_pot = curr_file_size / np.sum(all_file_sizes) * all_mc_pot

pot_weight = data_pot / curr_mc_pot


In [ ]:
# from data_files/CombinedFunctions_from_Fortran/CombinedTypes.h
particle_types = {
    1: "gamma",
    2: "positron",
    3: "electron",
    4: "neutrino",
    5: "mu+",
    6: "mu-",
    7: "pi0",
    8: "pi+",
    9: "pi-",
    10: "K0L",
    11: "K+",
    12: "K-",
    13: "neutron",
    14: "proton",
    15: "anti-proton",
    16: "K0S",
    17: "eta",
    18: "lambda",
    19: "sigma+",
    20: "sigma0",
    21: "sigma-",
    22: "xi0",
    23: "xi-",
    24: "omega-",
    25: "anti-neutron",
    26: "anti-lambda",
    27: "anti-sigma-",
    28: "anti-sigma0",
    29: "anti-sigma+",
    30: "anti-xi0",
    31: "anti-xi+",
    32: "anti-omega+",
    45: "deuteron",
    46: "triton",
    47: "alpha",
    48: "geantino",
    49: "He3",
    50: "Cerenkov"
}


In [ ]:
df = f["MiniBooNE_CCQE"].arrays(["EventNumber", "Weight", "PassOsc", "RecoEnuQE", "Energy", "CosTheta", "TrueEnergy", "NuType", "NFSP", "FSPType", "MomX", "MomY", "MomZ", "MomT"], library="pd")

# from data_files/SomeMiniBooNEDetails_on_files_v2.pdf
nu_types = {
    1: "numu",
    2: "numubar",
    3: "nue",
    4: "nuebar"
}
df["nu_type"] = df["NuType"].apply(lambda x: nu_types[x])
df.drop(columns=["NuType"], inplace=True)

df

In [ ]:
# from https://rtayloe.pages.iu.edu/MB/data-releases/nue2020/numode.html
nue_ccqe_bin_edges = [200.0, 300.0, 375.0, 475.0, 550.0, 675.0, 800.0, 950.0, 1100.0, 1250.0, 1500.0, 3000.0]
nue_ccqe_bin_centers = [(nue_ccqe_bin_edges[i] + nue_ccqe_bin_edges[i+1]) / 2. for i in range(len(nue_ccqe_bin_edges) - 1)]
nue_ccqe_bin_centers[-1] = 1600
nue_ccqe_bin_widths = [nue_ccqe_bin_edges[i+1] - nue_ccqe_bin_edges[i] for i in range(len(nue_ccqe_bin_edges) - 1)]

nue_ccqe_sel_df = df.query("PassOsc == True")

all_counts = np.histogram(nue_ccqe_sel_df["RecoEnuQE"] * 1000., weights=nue_ccqe_sel_df["Weight"] * pot_weight, bins=nue_ccqe_bin_edges)[0]
counts_per_mev = all_counts / nue_ccqe_bin_widths

plt.rcParams.update({'font.size': 16})

plt.figure(figsize=(10, 6))
plt.hist(nue_ccqe_bin_centers, bins=nue_ccqe_bin_edges, weights=counts_per_mev)
plt.xlabel(r"$E_{\nu}^{\mathrm{QE}}$ (MeV)")
plt.ylabel(r"Count per MeV")
plt.xlim(200, 1700)
plt.title("MiniBooNE Nue CCQE Selection\n100k MC Events, approx. weighted to 18.75e20 POT")
plt.show()

